# Parametric types and arrays

Arrays are a good example to get some idea of the Julian way of defining interfaces.

## Defining arrays

Defining arrays is rather intuitive, for example:

In [ ]:
v = [1, 2, 3, 4]    # An integer (column) vector

In [ ]:
f = [1.2, 3.4, 21, π]  # A float vector ... note the implicit conversion

In [ ]:
s = ["abc", "def", "ghi"]  # A string vector

The default `Vector` type is 1-based in Julia, and so are `Matrix` and `Array` (see below). Non-1-based arrays can be defined with the package [OffsetArrays.jl](https://juliaarrays.github.io/OffsetArrays.jl/stable/). It is good practice not to assume an array to be 1-based in one's own code unless one has constructed it oneself.)

In [ ]:
s[1]

The special arguments `begin` and `end` refer to the first and last element, respectively.

In [ ]:
s[begin+1]

In [ ]:
s[end-2]

In [ ]:
v = [1; 2; 3; 4]    # An equivalent notation for a (column) vector

In [ ]:
a = [1; 2;; 3; 4]    # A matrix

In [ ]:
[1 3; 2 4]    # An equivalent notation

In [ ]:
b = [-1; -2;; -3; -4;;; -5; -6;; -7; -8]    # A 3-dimensional array

Matrices and higher-dimensional arrays can alternatively be indexed linearly:

In [ ]:
@show b[2, 1, 2] b[6];

**Comprehensions** also work in Julia:

In [ ]:
e = rand(Int8, 2, 3)  # Matrix with random `Int8` values

In [ ]:
[2*x for x in e]

One can add an `if` clause to the comprehension. The shape of the matrix is (necessarily) lost in that case.

In [ ]:
[2*x for x in e if x > 0]

## Parametrized types

The type of arrays is `Array{T, N}`, where `T` is any type and `N` is the number of dimensions. This type is an example of a **parametric type**, i.e. a type, which itself is parametrised by other values or types. For convenience:

In [ ]:
Vector{String} === Array{String, 1}

In [ ]:
Matrix{Int} === Array{Int, 2}

Arrays do not need to be of one type only, for example ...

In [ ]:
c = [1, 3.4, "abc"]   # A mixed array

and we can explicitly fix their types

In [ ]:
d = Float32[1, 3.4, 4]

### Tuples

Another example of a parametric type is the `Tuple`. (It sometimes follows special rules, however.)

In [ ]:
(1, 2.0, "3")

In [ ]:
typeof((1, 2.0, "3"))

The brackets around tuples can often be omitted:

In [ ]:
1, 2.0, "3"

### Type parameters in function signatures

Since `Vector{Real} === Array{Real,1}` one would naively define

In [ ]:
myfunc(v::Vector{Real}) = "Got a real vector!"

However ...

In [ ]:
myfunc([1.0, 2.0, 3.0])

Why is this?

Note although we have

In [ ]:
Float64 <: Real

parametric types have the following (perhaps somewhat counterintuitive) property

In [ ]:
Vector{Float64} <: Vector{Real}

In [ ]:
[1.0, 2.0, 3.0] isa Vector{Real}

How can we understand the behavior above? The crucial point is that `Vector{Real}` is a **concrete** container type despite the fact that `Real` is an abstract type. Specifically, it describes a **heterogeneous** vector of values that individually can be of any type `T <: Real`.

In [ ]:
isconcretetype(Vector{Real})

In [ ]:
[1.0, 2.0]

In [ ]:
Real[1, 2.2, 13f0]

As we have learned above, concrete types are the leaves of the type tree and **cannot** have any subtypes. Hence it is only consistent to have...

In [ ]:
Vector{Float64} <: Vector{Real}

What we often actually *mean* when writing `myfunc(v::Vector{Real}) = ...` is

In [ ]:
myfunc(v::Vector{T}) where T <: Real = "I'm a real vector!"

Side remark: This could be shorter expressed as `myfunc(v::Vector{<:Real})`. If one uses this form, however, then one cannot use the element type `T` in the function body (well, not without calling some function).

In [ ]:
myfunc([1.0, 2.0, 3.0])

It works! But what does it mean exactly? First of all, we see that

In [ ]:
Vector{Float64} <: Vector{T} where T <: Real

Here, `Vector{T} where T <: Real` describes the **set** of concrete `Vector` types whose elements are of any specific single type `T` that is a subtype of `Real`.

Think of it as representing `{{ Vector{Float64}, Vector{Int64}, Vector{Int32}, Vector{AbstractFloat}, ... }}`.

In [ ]:
Vector{Int64} <: Vector{T} where T <: Real

In [ ]:
Vector{AbstractFloat} <: Vector{T} where T <: Real

In [ ]:
[1.0, 2.0, 3.0] isa Vector{T} where T <: Real

We can also use the `where` notation to write out our naive `Vector{Real}` from above in a more explicit way. The operator `===` tests if the two arguments are identical (and not just equal). This is sometimes called "égal".

In [ ]:
Vector{Real} === Vector{T where T <: Real}

This is because

In [ ]:
Real === T where T <: Real

Note that the crucial difference is the position of the `where T<:Real` piece, i.e. whether it is inside or outside of the curly braces.

In [ ]:
Vector{T where T<:Real} <: Vector{T} where T <: Real

In [ ]:
(Vector{T} where T<:Real) <: Vector{T where T <: Real}

## Basic functions for arrays

In [ ]:
a = rand(Int8, 4, 5)

In [ ]:
ndims(a)    # Get the number of dimensions

In [ ]:
eltype(a)   # Get the type of the array elements

In [ ]:
length(a)   # Return the number of elements

In [ ]:
size(a)     # Get the size of the array -- note that this doesn't imply that the array is 1-based

In [ ]:
size(a, 1)  # Get the size along an axis -- again, this doesn't imply that the array is 1-based

In [ ]:
axes(a)     # Get the axes as ranges (see below), also in the form `axes(A, 1)`

In [ ]:
reshape(a, 2, 5, 2)   # Return an array with the shape changed

In [ ]:
a[:, 1]     # First column of a. This is an expensive oeration because it creates a new `Vector`

In [ ]:
a[2, :]     # Second row (again copied)

In [ ]:
b = view(a, :, 1)   # The same without copying. This is much faster, but modifying B will modify A, too.

In [ ]:
b[2] = -1
a

The same view creating with the macro `@view`. The macro rewrites it to the form above before it is evaluated.

In [ ]:
@view a[:, 1]

In [ ]:
b == a[:, 1]

## Arrays and loops

Arrays are "iterable". This means that `for` can directly iterate over its elements:

In [ ]:
function mysum(a::AbstractArray{T}) where T <: Number
    s = zero(T)   # the zero element of T. Alternatively: s = zero(eltype(a))
    for x in a
        s += x
    end
    return s
end

In [ ]:
c = [1 2; 3 4]
mysum(c), sum(c)

In [ ]:
function myisequal(v::AbstractVector, w::AbstractVector)
    if axes(v) != axes(w)
        return false
    end
    # shorter form: axes(v) != axes(w) && return false
    
    for i in axes(v)
        v[i] == w[i] || return false
    end
    
    return true
end

In [ ]:
v = [1, 2, 3]
myisequal(v, v)

In [ ]:
function myisequal(v::AbstractMatrix, w::AbstractMatrix)
    axes(v) == axes(w) || return false
    
    for i1 in axes(v, 1), i2 in axes(v, 2)
        v[i1, i2] == w[i1, i2] || return false
    end
    
    return true
end

In [ ]:
a = [1 2; 3 4]
myisequal(a, a)

Linear indexing together with `eachindex` provides an easy way to write code for arrays of any dimension:

In [ ]:
function myisequal(v::AbstractArray, w::AbstractArray)
    axes(v) == axes(w) || return false
    
    for i in eachindex(v, w)
        v[i] == w[i] || return false
    end
    
    return true
end

In [ ]:
b = rand(Int8, 1, 2, 3)

In [ ]:
myisequal(b, b)

Here is another version, this time using `all` and an anonymous function:

In [ ]:
myisequal2(v::AbstractArray, w::AbstractArray) = axes(v) == axes(w) && all(i -> v[i] == w[i], eachindex(v, w))

In [ ]:
myisequal2(b, b)

Here is an even shorter version using vectorized comparison (see below). It is slower, however, because `.==` creates a new array.

In [ ]:
myisequal3(v::AbstractArray, w::AbstractArray) = axes(v) == axes(w) && all(v .== w)

Julia provides the `push!`, `pushfirst!`, `insert!` and `append!` functions to add additional elements to an existing vector and `pop!`, `popfirst!`, `deleteat!` to remove elements. For example:

In [ ]:
a = Vector{Float64}()  # Create an empty Float64 array
a = Float64[]          # Shorter notation. WARNING: [] means Any[], and that is really slow!

In [ ]:
push!(a, 4)

In [ ]:
a

In [ ]:
append!(a, [5, 6, 7])

In [ ]:
popfirst!(a)
a

Notice, that the `!` is part of the name of the function. In Julia the `!` is a convention to indicate that the respective function *mutates* the content of at least one of its arguments.

Very helpful functions as we will see are:

- `zero`, which allocates an array of zeros of the same element type

In [ ]:
a = rand(Float32, 3, 4)
zero(a)

- `similar`, which returns an uninitialised array, which is similar to the passed array. This means that by default array type, element type and size are all kept.

In [ ]:
similar(a)

- One may also change these parameters easily:

In [ ]:
similar(a, (3, 2))            # Keep element type and array type

In [ ]:
similar(a, Float64)           # Change element type

In [ ]:
similar(a, Float64, (1, 2))   # Change element type and shape

## Vector operations and vectorised operations

Array addition (`+`, `-`) and scalar multiplication are directly available on arrays (of any dimension):

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]

In [ ]:
x + 2.0y

For element-wise operations the vectorisation syntax is used:

In [ ]:
x .* y  # elementwise multiplication

In [ ]:
x .^ y  # Elementwise exponentiation

Note, that the `.`-syntax continues to *all* functions in Julia. That includes base Julia ...

In [ ]:
sqrt.(cos.(2π .* x) .+ sin.(2π * x))

In [ ]:
@. sqrt(cos(2π * x) + sin(2π * x))

... custom functions ...

In [ ]:
myfun(x) = x * x + x
myfun.(y)

... and may be easily chained 

In [ ]:
@. exp(cos(x^2))

### Exercises
- Create the following arrays using Julia code:
$$\left(\begin{array}{ccccc}
   2&2&2&2&2 \\
   2&2&2&2&2 \\
   2&2&2&2&2 \\
   \end{array}\right) \qquad
   \left(\begin{array}{cccc}
   0.1&0.5&0.9&1.3\\
   0.2&0.6&1.0&1.4\\
   0.3&0.7&1.1&1.5\\
   0.4&0.8&1.2&1.6\\
   \end{array}\right)
$$

- Write your own function for multiplication of a `Vector`/`Matrix`/`Array` by a scalar. Can you extend it to `AbstractVector` etc.?

##### More details on Arrays
- https://docs.julialang.org/en/v1/manual/arrays/
- https://docs.julialang.org/en/v1/base/arrays/
- https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1

## `UnitRange` as an `AbstractArray`

In [ ]:
x = 1:30

In [ ]:
typeof(x)

In [ ]:
typeof(x) <: AbstractVector{Int}

Because it is a subtype of `AbstractVector`, we can do vector-like things with it. (It should basically behave like a vector, meaning that it implements the "vector interface".)

In [ ]:
x[3]

In [ ]:
size(x)

In [ ]:
eltype(x)

However, it's not implemented like a regular `Array` at all. In fact, it's just two numbers! We can see this by looking at it's fields:

In [ ]:
fieldnames(typeof(x))

or just by inspecting the source code

In [ ]:
@edit UnitRange(1, 30)

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Julia is pretty smart here, for example:

In [ ]:
(1:10) .+ 3

Allocating memory is typically costly. The function `collect` converts an `AbstractVector` (and more general objects called "iterators") to a `Vector`.

In [ ]:
collect(1:10)

In [ ]:
@time collect(1:10000000);

But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [ ]:
@time 1:10000000;

Yet, in code they *act* the same way.

## Takeaways

- Parametric types ar types that by themselves have parameters (e.g. `Vector{Float64}`). The notation `T where T <: SuperType` exists to denote sets of types.
- Arrays are deeply built into Julia
- Julia has many clever Array types (like `UnitRange`) to speed up array operations.